In [10]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import seaborn as sns
import lightgbm as lgb
from sklearn.model_selection import KFold
import warnings
import time
import sys
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import BayesianRidge
warnings.simplefilter(action='ignore', category=FutureWarning)
import gc

In [11]:
print('tung beo')

tung beo


In [12]:
#this is used to reduce the memory usage of the dataframe 
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [13]:
#Read the csv 
new_transactions = pd.read_csv('../input/elo-merchant-category-recommendation/new_merchant_transactions.csv', parse_dates=['purchase_date'])
historical_transactions = pd.read_csv('../input/elo-merchant-category-recommendation/historical_transactions.csv', parse_dates=['purchase_date'])
df_merchant = pd.read_csv('../input/elo-merchant-category-recommendation/merchants.csv')

def aggregate_transactions(history):
    '''
    Parameters 
    -----------
    history: historical transaction of dataframe
    
    output:
    ----------
    dataframe groupby card_id and aggregated on multiple function
    '''
    
    history.loc[:, 'purchase_date'] = pd.DatetimeIndex(history['purchase_date']).\
                                      astype(np.int64) * 1e-9
    
    
    agg_func = {
        'category_1': ['sum', 'mean'],
        'category_2_1.0': ['mean'],
        'category_2_2.0': ['mean'],
        'category_2_3.0': ['mean'],
        'category_2_4.0': ['mean'],
        'category_2_5.0': ['mean'],
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        'merchant_id': ['nunique'],
        '2w_purchase':['sum','mean','max','min','std'],
        '1m_purchase':['sum','mean','max','min','std'],
        '3m_purchase':['sum','mean','max','min','std'],
        '6m_purchase':['sum','mean','max','min','std'],
        '1y_purchase':['sum','mean','max','min','std'],
#         'merchant_category_id': ['nunique'],
#         'state_id': ['nunique'],
#         'city_id': ['nunique'],
#         'subsector_id': ['nunique'],
        'purchase_amount': ['sum',
                            'mean',
                            'max',
                            'min',
                            'std'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'purchase_month': ['mean', 'max', 'min', 'std'],
        'purchase_date': [np.ptp, 'min', 'max'],
        'month_lag': ['min', 'max','std','sum',]
        }
    #groupby card id 
    agg_history = history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    df = (history.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    #merge the two dataframe together 
    agg_history = pd.merge(df, agg_history, on='card_id', how='left')
    
    return agg_history

gc.collect()
def binarize(df):
    """
    Parameters
    -----------
    df : input dataframe 
    
    Return 
    ----------
    dataframe with column authorized_flag and category_1 ,mapped to either 1 or 0 """
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    return df

historical_transactions = binarize(historical_transactions)
new_transactions = binarize(new_transactions)

In [14]:
%%time
def read_data(input_file):
    #change the column type to date type
    #calculate the elapsed time 
    df = pd.read_csv(input_file)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['elapsed_time'] = (datetime.date(2018, 2, 1) - df['first_active_month'].dt.date).dt.days
    return df

train = read_data('../input/elo-merchant-category-recommendation/train.csv')
test = read_data('../input/elo-merchant-category-recommendation/test.csv')

#take out the target column in the training data 
target = train['target']
del train['target']
gc.collect()

CPU times: user 1.72 s, sys: 12 ms, total: 1.74 s
Wall time: 1.75 s


## **Feature Engineering**

In [15]:
%%time
#create one-hot-encoding dataframe for category_2 and category_3
historical_transactions = pd.get_dummies(historical_transactions, columns=['category_2', 'category_3'])
new_transactions = pd.get_dummies(new_transactions, columns=['category_2', 'category_3'])

#reduce memory usage
historical_transactions = reduce_mem_usage(historical_transactions)
new_transactions = reduce_mem_usage(new_transactions)
df_merchant=reduce_mem_usage(df_merchant)



Mem. usage decreased to 1304.89 Mb (54.8% reduction)
Mem. usage decreased to 84.24 Mb (56.7% reduction)
Mem. usage decreased to 30.32 Mb (46.0% reduction)
CPU times: user 11.4 s, sys: 5.06 s, total: 16.5 s
Wall time: 16.6 s


In [16]:
df_merchant['abs_numerical_1'] = abs(df_merchant['numerical_1'])+1
cols = ['merchant_id','abs_numerical_1']
df_trick = df_merchant[cols]
df_trick.head()


#     historical_transactions['today_date'] = pd.to_datetime('2018-2-1')
#     historical_transactions['day_diff'] = (historical_transactions['today_date'] - historical_transactions['purchase_date']).dt.days
#     historical_transactions = historical_transactions(right = df_trick , how='left', on = 'merchant_id')
#     historical_transactions['2w_purchase'] = np.where(historical_transactions['day_diff'] <= 14,historical_transactions['purchase_amount'],0)
#     historical_transactions['1m_purchase'] = np.where(historical_transactions['day_diff'] <= 30,historical_transactions['purchase_amount'],0)
#     historical_transactions['3m_purchase'] = np.where(historical_transactions['day_diff'] <= 90,historical_transactions['purchase_amount'],0)
#     historical_transactions['6m_purchase'] = np.where(historical_transactions['day_diff'] <= 180,historical_transactions['purchase_amount'],0)
#     historical_transactions['1y_purchase'] = np.where(historical_transactions['day_diff'] <= 365,historical_transactions['purchase_amount'],0)

,merchant_id,abs_numerical_1
0,M_ID_838061e48c,1.057617
1,M_ID_9339d880ad,1.057617
2,M_ID_e726bbae1e,1.057617
3,M_ID_a70e9c5f81,1.057617
4,M_ID_64456c37ce,1.057617


In [17]:
historical_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C
0,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,16,37,1,0,0,0,0,1,0,0
1,1,C_ID_4e6213e9bc,88,0,0,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,16,16,1,0,0,0,0,1,0,0
2,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,16,37,1,0,0,0,0,1,0,0
3,1,C_ID_4e6213e9bc,88,0,0,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,16,34,1,0,0,0,0,1,0,0
4,1,C_ID_4e6213e9bc,88,0,0,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,16,37,1,0,0,0,0,1,0,0


In [18]:

#Amount of purchase in the last month
# for df in [historical_transactions,new_transactions]:
def purchase_lapse(df):
    df['today_date'] = pd.to_datetime('2018-2-1')
    df['day_diff'] = (df['today_date'] - df['purchase_date']).dt.days
    df = df.merge(right = df_trick , how='left', on = 'merchant_id')
    df['purchase_amount'] = df['purchase_amount']*df['abs_numerical_1']
    df['2w_purchase'] = np.where(df['day_diff'] <= 14,df['purchase_amount'],0)
    df['1m_purchase'] = np.where(df['day_diff'] <= 30,df['purchase_amount'],0)
    df['3m_purchase'] = np.where(df['day_diff'] <= 90,df['purchase_amount'],0)
    df['6m_purchase'] = np.where(df['day_diff'] <= 180,df['purchase_amount'],0)
    df['1y_purchase'] = np.where(df['day_diff'] <= 365,df['purchase_amount'],0)
    return df

historical_transactions = purchase_lapse(historical_transactions)
new_transactions = purchase_lapse(new_transactions)


In [19]:
agg_fun = {'authorized_flag': ['sum', 'mean']}

#groupby ohe dataframe by card_id
auth_mean = historical_transactions.groupby(['card_id']).agg(agg_fun)
auth_mean.columns = ['_'.join(col).strip() for col in auth_mean.columns.values]
auth_mean.reset_index(inplace=True)

authorized_transactions = historical_transactions[historical_transactions['authorized_flag'] == 1]
historical_transactions = historical_transactions[historical_transactions['authorized_flag'] == 0]
gc.collect()

206

In [20]:
new_transactions

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C,today_date,day_diff,abs_numerical_1,2w_purchase,1m_purchase,3m_purchase,6m_purchase,1y_purchase
0,1,C_ID_415bb3a509,107,0,1,307,M_ID_b0c793002c,1,-0.584473,2018-03-11 14:57:36,9,19,1,0,0,0,0,0,1,0,2018-02-01,-39,1.047852,-0.584473,-0.584473,-0.584473,-0.584473,-0.584473
1,1,C_ID_415bb3a509,140,0,1,307,M_ID_88920c89e8,1,-0.587891,2018-03-19 18:53:37,9,19,1,0,0,0,0,0,1,0,2018-02-01,-47,1.032227,-0.587891,-0.587891,-0.587891,-0.587891,-0.587891
2,1,C_ID_415bb3a509,330,0,1,507,M_ID_ad5237ef6b,2,-0.645508,2018-04-26 14:08:44,9,14,1,0,0,0,0,0,1,0,2018-02-01,-85,1.170898,-0.645508,-0.645508,-0.645508,-0.645508,-0.645508
3,1,C_ID_415bb3a509,-1,1,1,661,M_ID_9e84cda3b1,1,-0.704102,2018-03-07 09:43:21,-1,8,0,0,0,0,0,0,1,0,2018-02-01,-35,1.047852,-0.704102,-0.704102,-0.704102,-0.704102,-0.704102
4,1,C_ID_ef55cf8d4b,-1,1,1,166,M_ID_3c86fa3831,1,-0.697754,2018-03-22 21:07:53,-1,29,0,0,0,0,0,0,1,0,2018-02-01,-50,1.057617,-0.697754,-0.697754,-0.697754,-0.697754,-0.697754
5,1,C_ID_ef55cf8d4b,231,0,1,367,M_ID_8874615e00,2,-0.637695,2018-04-02 12:53:28,9,16,1,0,0,0,0,0,1,0,2018-02-01,-61,1.007812,-0.637695,-0.637695,-0.637695,-0.637695,-0.637695
6,1,C_ID_ef55cf8d4b,69,0,1,333,M_ID_6d061b5ddc,1,5.410156,2018-03-28 19:50:19,9,10,1,0,0,0,0,0,1,0,2018-02-01,-56,1.027344,5.410156,5.410156,5.410156,5.410156,5.410156
7,1,C_ID_ef55cf8d4b,231,0,1,307,M_ID_df1e022f41,2,-0.585449,2018-04-05 08:06:52,9,19,1,0,0,0,0,0,1,0,2018-02-01,-64,1.057617,-0.585449,-0.585449,-0.585449,-0.585449,-0.585449
8,1,C_ID_ef55cf8d4b,69,0,1,278,M_ID_d15eae0468,2,-0.619629,2018-04-07 18:37:40,9,37,1,0,0,0,0,0,1,0,2018-02-01,-66,1.038086,-0.619629,-0.619629,-0.619629,-0.619629,-0.619629
9,1,C_ID_ef55cf8d4b,69,0,1,437,M_ID_5f9bffd028,1,-0.630371,2018-03-17 18:10:41,9,15,1,0,0,0,0,0,1,0,2018-02-01,-45,1.038086,-0.630371,-0.630371,-0.630371,-0.630371,-0.630371


In [21]:
historical_transactions.head()

,authorized_flag,card_id,city_id,category_1,installments,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,state_id,subsector_id,category_2_1.0,category_2_2.0,category_2_3.0,category_2_4.0,category_2_5.0,category_3_A,category_3_B,category_3_C,today_date,day_diff,abs_numerical_1,2w_purchase,1m_purchase,3m_purchase,6m_purchase,1y_purchase
117,0,C_ID_4e6213e9bc,88,0,0,842,M_ID_22c9cfa265,-10,-9.763108,2017-04-07 12:58:09,16,37,1,0,0,0,0,1,0,0,2018-02-01,299,13.367188,0.000000,0.000000,0.000000,0.000000,-9.763108
134,0,C_ID_4e6213e9bc,88,0,0,367,M_ID_86ec983688,-5,-1.515419,2017-09-17 22:40:27,16,16,1,0,0,0,0,1,0,0,2018-02-01,136,2.093750,0.000000,0.000000,0.000000,-1.515419,-1.515419
152,0,C_ID_4e6213e9bc,88,0,0,367,M_ID_86ec983688,-5,-1.515419,2017-09-17 22:40:26,16,16,1,0,0,0,0,1,0,0,2018-02-01,136,2.093750,0.000000,0.000000,0.000000,-1.515419,-1.515419
174,0,C_ID_4e6213e9bc,333,0,0,605,M_ID_c2ae34c2ef,0,-0.731726,2018-02-20 10:57:50,9,2,1,0,0,0,0,1,0,0,2018-02-01,-20,1.101562,-0.731726,-0.731726,-0.731726,-0.731726,-0.731726
220,0,C_ID_4e6213e9bc,88,0,0,560,M_ID_e6d5ae8ea6,-7,-0.780661,2017-07-08 07:33:31,16,34,1,0,0,0,0,1,0,0,2018-02-01,207,1.057617,0.000000,0.000000,0.000000,0.000000,-0.780661


In [22]:
%%time
#create purchase month
historical_transactions['purchase_month'] = historical_transactions['purchase_date'].dt.month
authorized_transactions['purchase_month'] = authorized_transactions['purchase_date'].dt.month
new_transactions['purchase_month'] = new_transactions['purchase_date'].dt.month
gc.collect()

CPU times: user 1.85 s, sys: 0 ns, total: 1.85 s
Wall time: 1.86 s


In [23]:
%%time
def aggregate_transactions(history):
    '''
    Parameters 
    -----------
    history: historical transaction of dataframe
    
    
    output:
    ----------
    dataframe groupby card_id and aggregated on multiple function
    '''
    
    history.loc[:, 'purchase_date'] = pd.DatetimeIndex(history['purchase_date']).\
                                      astype(np.int64) * 1e-9
    
    
    agg_func = {
        'category_1': ['sum', 'mean'],
        'category_2_1.0': ['mean'],
        'category_2_2.0': ['mean'],
        'category_2_3.0': ['mean'],
        'category_2_4.0': ['mean'],
        'category_2_5.0': ['mean'],
        'category_3_A': ['mean'],
        'category_3_B': ['mean'],
        'category_3_C': ['mean'],
        'merchant_id': ['nunique'],
        '2w_purchase':['sum','mean','max','min','std'],
        '1m_purchase':['sum','mean','max','min','std'],
        '3m_purchase':['sum','mean','max','min','std'],
        '6m_purchase':['sum','mean','max','min','std'],
        '1y_purchase':['sum','mean','max','min','std'],
#         'merchant_category_id': ['nunique'],
#         'state_id': ['nunique'],
#         'city_id': ['nunique'],
#         'subsector_id': ['nunique'],
        'purchase_amount': ['sum',
                            'mean',
                            'max',
                            'min',
                            'std'],
        'installments': ['sum', 'mean', 'max', 'min', 'std'],
        'purchase_month': ['mean', 'max', 'min', 'std'],
        'purchase_date': [np.ptp, 'min', 'max'],
        'month_lag': ['min', 'max','std','sum',]
        }
    #groupby card id 
    agg_history = history.groupby(['card_id']).agg(agg_func)
    agg_history.columns = ['_'.join(col).strip() for col in agg_history.columns.values]
    agg_history.reset_index(inplace=True)
    
    df = (history.groupby('card_id')
          .size()
          .reset_index(name='transactions_count'))
    
    #merge the two dataframe together 
    agg_history = pd.merge(df, agg_history, on='card_id', how='left')
    
    return agg_history

gc.collect()

CPU times: user 52 ms, sys: 0 ns, total: 52 ms
Wall time: 52.6 ms


In [24]:
%%time
history = aggregate_transactions(historical_transactions)
history.columns = ['hist_' + c if c != 'card_id' else c for c in history.columns]
gc.collect()
history[:5].head()


CPU times: user 50.6 s, sys: 32 ms, total: 50.6 s
Wall time: 50.7 s


In [25]:
history[:5].head()

,card_id,hist_transactions_count,hist_category_1_sum,hist_category_1_mean,hist_category_2_1.0_mean,hist_category_2_2.0_mean,hist_category_2_3.0_mean,hist_category_2_4.0_mean,hist_category_2_5.0_mean,hist_category_3_A_mean,hist_category_3_B_mean,hist_category_3_C_mean,hist_merchant_id_nunique,hist_2w_purchase_sum,hist_2w_purchase_mean,hist_2w_purchase_max,hist_2w_purchase_min,hist_2w_purchase_std,hist_1m_purchase_sum,hist_1m_purchase_mean,hist_1m_purchase_max,hist_1m_purchase_min,hist_1m_purchase_std,hist_3m_purchase_sum,hist_3m_purchase_mean,hist_3m_purchase_max,hist_3m_purchase_min,hist_3m_purchase_std,hist_6m_purchase_sum,hist_6m_purchase_mean,hist_6m_purchase_max,hist_6m_purchase_min,hist_6m_purchase_std,hist_1y_purchase_sum,hist_1y_purchase_mean,hist_1y_purchase_max,hist_1y_purchase_min,hist_1y_purchase_std,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_std,hist_installments_sum,hist_installments_mean,hist_installments_max,hist_installments_min,hist_installments_std,hist_purchase_month_mean,hist_purchase_month_max,hist_purchase_month_min,hist_purchase_month_std,hist_purchase_date_ptp,hist_purchase_date_min,hist_purchase_date_max,hist_month_lag_min,hist_month_lag_max,hist_month_lag_std,hist_month_lag_sum
0,C_ID_00007093c1,35,4.0,0.114286,0.000000,0.000000,0.885714,0.0,0.000000,0.000000,0.828571,0.171429,11,-2.608193,-0.074520,0.0,-0.806261,0.214087,-3.779852,-0.107996,0.0,-0.806261,0.244587,-6.817972,-0.194799,0.000000,-2.458895,0.469534,-8.439823,-0.241138,3.585176,-2.458895,0.841686,-19.372437,-0.553498,3.585176,-2.600237,0.934593,-19.372437,-0.553498,3.585176,-2.600237,0.934593,45,1.285714,3,1,0.667367,5.914286,12,1,3.071419,28858113.0,1.489250e+09,1.518108e+09,-11,0,3.535415,-211.0
1,C_ID_0001238066,3,0.0,0.000000,0.333333,0.000000,0.000000,0.0,0.666667,0.000000,1.000000,0.000000,2,-0.607131,-0.202377,0.0,-0.607131,0.350527,-0.607131,-0.202377,0.0,-0.607131,0.350527,-1.845863,-0.615288,-0.607131,-0.619366,0.007064,-1.845863,-0.615288,-0.607131,-0.619366,0.007064,-1.845863,-0.615288,-0.607131,-0.619366,0.007064,-1.845863,-0.615288,-0.607131,-0.619366,0.007064,3,1.000000,1,1,0.000000,8.666667,12,2,5.773503,3609150.0,1.514660e+09,1.518269e+09,-2,0,1.154701,-4.0
2,C_ID_0001506ef0,4,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.750000,0.250000,0.000000,4,-1.500575,-0.375144,0.0,-0.751157,0.433179,-1.500575,-0.375144,0.0,-0.751157,0.433179,-2.425381,-0.606345,0.000000,-0.924807,0.412518,-2.425381,-0.606345,0.000000,-0.924807,0.412518,-6.587753,-1.646938,-0.749417,-4.162372,1.678973,-6.587753,-1.646938,-0.749417,-4.162372,1.678973,1,0.250000,1,0,0.500000,5.500000,12,2,4.725816,22098875.0,1.496772e+09,1.518871e+09,-8,0,3.785939,-10.0
3,C_ID_0001793786,27,2.0,0.074074,0.111111,0.296296,0.111111,0.0,0.000000,0.814815,0.185185,0.000000,17,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-15.039488,-0.557018,0.000000,-3.334431,0.958001,-28.728289,-1.064011,1.307841,-4.025151,1.349841,-28.728289,-1.064011,1.307841,-4.025151,1.349841,5,0.185185,1,0,0.395847,6.481481,9,3,2.375684,16780236.0,1.488636e+09,1.505416e+09,-7,-1,2.375684,-95.0
4,C_ID_000183fdda,7,0.0,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.000000,0.428571,0.571429,4,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.499343,-0.071335,0.540302,-0.828979,0.590313,-0.499343,-0.071335,0.540302,-0.828979,0.590313,-0.499343,-0.071335,0.540302,-0.828979,0.590313,19,2.714286,6,1,2.288689,8.285714,10,8,0.755929,6701589.0,1.502099e+09,1.508801e+09,-6,-4,0.755929,-40.0


In [26]:
%%time
#authorized transaction
authorized = aggregate_transactions(authorized_transactions)
authorized.columns = ['auth_' + c if c != 'card_id' else c for c in authorized.columns]
gc.collect()

CPU times: user 1min 59s, sys: 4.5 s, total: 2min 3s
Wall time: 2min 3s


In [27]:
authorized[:5]

,card_id,auth_transactions_count,auth_category_1_sum,auth_category_1_mean,auth_category_2_1.0_mean,auth_category_2_2.0_mean,auth_category_2_3.0_mean,auth_category_2_4.0_mean,auth_category_2_5.0_mean,auth_category_3_A_mean,auth_category_3_B_mean,auth_category_3_C_mean,auth_merchant_id_nunique,auth_2w_purchase_sum,auth_2w_purchase_mean,auth_2w_purchase_max,auth_2w_purchase_min,auth_2w_purchase_std,auth_1m_purchase_sum,auth_1m_purchase_mean,auth_1m_purchase_max,auth_1m_purchase_min,auth_1m_purchase_std,auth_3m_purchase_sum,auth_3m_purchase_mean,auth_3m_purchase_max,auth_3m_purchase_min,auth_3m_purchase_std,auth_6m_purchase_sum,auth_6m_purchase_mean,auth_6m_purchase_max,auth_6m_purchase_min,auth_6m_purchase_std,auth_1y_purchase_sum,auth_1y_purchase_mean,auth_1y_purchase_max,auth_1y_purchase_min,auth_1y_purchase_std,auth_purchase_amount_sum,auth_purchase_amount_mean,auth_purchase_amount_max,auth_purchase_amount_min,auth_purchase_amount_std,auth_installments_sum,auth_installments_mean,auth_installments_max,auth_installments_min,auth_installments_std,auth_purchase_month_mean,auth_purchase_month_max,auth_purchase_month_min,auth_purchase_month_std,auth_purchase_date_ptp,auth_purchase_date_min,auth_purchase_date_max,auth_month_lag_min,auth_month_lag_max,auth_month_lag_std,auth_month_lag_sum
0,C_ID_00007093c1,114,24.0,0.210526,0.000000,0.000000,0.780702,0.0,0.008772,0.0,0.842105,0.157895,28,-5.913467,-0.051873,0.004672,-1.051259,0.184636,-19.061174,-0.167203,0.004672,-7.525646,0.787550,-35.272644,-0.309409,0.047582,-7.525646,0.924780,-59.062454,-0.518092,2.870243,-7.525646,1.079370,-164.286087,-1.441106,2.870243,-11.429580,2.306104,-164.286087,-1.441106,2.870243,-11.429580,2.306104,147,1.289474,6,1,0.795159,6.517544,12,1,3.371490,32627654.0,1.487081e+09,1.519708e+09,-12,0,3.441495,-661.0
1,C_ID_0001238066,120,2.0,0.016667,0.783333,0.000000,0.000000,0.0,0.150000,0.0,0.708333,0.266667,65,-210.680496,-1.755671,0.812351,-86.576363,11.101221,-233.973663,-1.949780,0.812351,-86.576363,11.178206,-623.906372,-5.199220,0.812351,-86.576363,18.734945,-889.466675,-7.412222,0.812351,-86.778114,22.611687,-889.466675,-7.412222,0.812351,-86.778114,22.611687,-889.466675,-7.412222,0.812351,-86.778114,22.611687,195,1.625000,10,-1,1.501050,7.275000,12,1,4.895483,13110825.0,1.506638e+09,1.519748e+09,-5,0,1.294218,-219.0
2,C_ID_0001506ef0,64,0.0,0.000000,0.062500,0.000000,0.937500,0.0,0.000000,1.0,0.000000,0.000000,28,-7.191096,-0.112361,2.074044,-6.385839,0.857748,-8.894343,-0.138974,2.074044,-6.385839,0.888193,-41.718681,-0.651854,2.074044,-10.838410,1.672008,-42.439651,-0.663120,2.074044,-10.838410,1.685545,-71.038536,-1.109977,2.074044,-10.838410,1.936746,-72.973701,-1.140214,2.074044,-10.838410,1.927021,0,0.000000,0,0,0.000000,7.046875,12,1,4.554536,34460275.0,1.484411e+09,1.518871e+09,-13,0,4.213045,-313.0
3,C_ID_0001793786,195,0.0,0.000000,0.071795,0.348718,0.061538,0.0,0.000000,1.0,0.000000,0.000000,114,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-29.979034,-0.153739,4.439522,-4.163296,0.747384,-60.312984,-0.309297,9.170553,-4.515928,1.327347,-62.969501,-0.322921,9.170553,-4.515928,1.352594,0,0.000000,0,0,0.000000,6.702564,10,1,2.289317,24487497.0,1.484994e+09,1.509481e+09,-9,0,2.289317,-643.0
4,C_ID_000183fdda,142,4.0,0.028169,0.084507,0.007042,0.873239,0.0,0.007042,0.0,0.739437,0.232394,71,-25.377922,-0.178718,1.219158,-3.062274,0.407161,-24.910986,-0.175429,5.751264,-3.062274,0.748805,-45.642822,-0.321428,5.751264,-3.062274,0.822687,-83.878273,-0.590692,5.751264,-3.088500,0.851161,-83.878273,-0.590692,5.751264,-3.088500,0.851161,-83.878273,-0.590692,5.751264,-3.088500,0.851161,250,1.760563,10,-1,2.076309,6.809859,12,1,4.519356,15148616.0,1.504444e+09,1.519592e+09,-5,0,1.792739,-325.0


In [28]:
%%time
#doing the same thing for the new transaction
new = aggregate_transactions(new_transactions)
new.columns = ['new_' + c if c != 'card_id' else c for c in new.columns]

gc.collect()

CPU times: user 52.4 s, sys: 52 ms, total: 52.4 s
Wall time: 52.4 s


In [29]:
new[:5]

,card_id,new_transactions_count,new_category_1_sum,new_category_1_mean,new_category_2_1.0_mean,new_category_2_2.0_mean,new_category_2_3.0_mean,new_category_2_4.0_mean,new_category_2_5.0_mean,new_category_3_A_mean,new_category_3_B_mean,new_category_3_C_mean,new_merchant_id_nunique,new_2w_purchase_sum,new_2w_purchase_mean,new_2w_purchase_max,new_2w_purchase_min,new_2w_purchase_std,new_1m_purchase_sum,new_1m_purchase_mean,new_1m_purchase_max,new_1m_purchase_min,new_1m_purchase_std,new_3m_purchase_sum,new_3m_purchase_mean,new_3m_purchase_max,new_3m_purchase_min,new_3m_purchase_std,new_6m_purchase_sum,new_6m_purchase_mean,new_6m_purchase_max,new_6m_purchase_min,new_6m_purchase_std,new_1y_purchase_sum,new_1y_purchase_mean,new_1y_purchase_max,new_1y_purchase_min,new_1y_purchase_std,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_max,new_purchase_amount_min,new_purchase_amount_std,new_installments_sum,new_installments_mean,new_installments_max,new_installments_min,new_installments_std,new_purchase_month_mean,new_purchase_month_max,new_purchase_month_min,new_purchase_month_std,new_purchase_date_ptp,new_purchase_date_min,new_purchase_date_max,new_month_lag_min,new_month_lag_max,new_month_lag_std,new_month_lag_sum
0,C_ID_00007093c1,3,0,0.000000,0.666667,0.000000,0.333333,0.0,0.000000,0.0,1.000000,0.000000,2,-2.09375,-0.697754,-0.687988,-0.710449,0.011581,-2.09375,-0.697754,-0.687988,-0.710449,0.011581,-2.09375,-0.697754,-0.687988,-0.710449,0.011581,-2.09375,-0.697754,-0.687988,-0.710449,0.011581,-2.09375,-0.697754,-0.687988,-0.710449,0.011581,-2.09375,-0.697754,-0.687988,-0.710449,0.011581,3,1.000000,1,1,0.000000,4.000000,4,4,0.000000,537024.0,1.522754e+09,1.523291e+09,2,2,0.000000,6.0
1,C_ID_0001238066,27,2,0.074074,0.777778,0.000000,0.000000,0.0,0.111111,0.0,0.814815,0.148148,25,-151.37500,-5.824219,-0.082031,-90.562500,19.390625,-151.37500,-5.824219,-0.082031,-90.562500,19.390625,-151.37500,-5.824219,-0.082031,-90.562500,19.390625,-151.37500,-5.824219,-0.082031,-90.562500,19.390625,-151.37500,-5.824219,-0.082031,-90.562500,19.390625,-151.37500,-5.824219,-0.082031,-90.562500,19.390625,43,1.592593,10,-1,2.042985,3.333333,4,3,0.480384,5195343.0,1.519923e+09,1.525118e+09,1,2,0.480384,36.0
2,C_ID_0001506ef0,2,0,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,1.0,0.000000,0.000000,1,-0.76709,-0.767090,-0.767090,-0.767090,NaN,-0.76709,-0.767090,-0.767090,-0.767090,NaN,-0.76709,-0.767090,-0.767090,-0.767090,NaN,-0.76709,-0.767090,-0.767090,-0.767090,NaN,-0.76709,-0.767090,-0.767090,-0.767090,NaN,-0.76709,-0.767090,-0.767090,-0.767090,NaN,0,0.000000,0,0,0.000000,3.000000,3,3,0.000000,471152.0,1.521239e+09,1.521710e+09,1,1,0.000000,2.0
3,C_ID_0001793786,31,0,0.000000,0.483871,0.258065,0.161290,0.0,0.032258,1.0,0.000000,0.000000,31,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,-91.25000,-2.943359,4.332031,-84.000000,15.179688,-91.25000,-2.943359,4.332031,-84.000000,15.179688,-91.25000,-2.943359,4.332031,-84.000000,15.179688,-91.25000,-2.943359,4.332031,-84.000000,15.179688,0,0.000000,0,0,0.000000,11.322581,12,11,0.475191,3981096.0,1.510761e+09,1.514742e+09,1,2,0.475191,41.0
4,C_ID_000183fdda,11,0,0.000000,0.000000,0.000000,1.000000,0.0,0.000000,0.0,0.545455,0.363636,11,-8.40625,-0.764160,-0.114319,-1.438477,0.361816,-8.40625,-0.764160,-0.114319,-1.438477,0.361816,-8.40625,-0.764160,-0.114319,-1.438477,0.361816,-8.40625,-0.764160,-0.114319,-1.438477,0.361816,-8.40625,-0.764160,-0.114319,-1.438477,0.361816,-8.40625,-0.764160,-0.114319,-1.438477,0.361816,16,1.454545,4,-1,1.293340,3.272727,4,3,0.467099,5106807.0,1.519994e+09,1.525100e+09,1,2,0.467099,14.0


In [30]:
%%time
def aggregate_per_month(history):
    '''
    Parameter:
    ----------
    history transaction dataframe
    
    Output:
    ---------
    Create a new dataframe aggregated per month
    order : groupby card_id + month_lag ----> groupby card_id 
    '''
    grouped = history.groupby(['card_id', 'month_lag'])

    agg_func = {
            'purchase_amount': ['count', 'sum', 'mean', 'min', 'max', 'std'],
            'installments': ['count', 'sum', 'mean', 'min', 'max', 'std'],
            }
    #group by and create new column
    intermediate_group = grouped.agg(agg_func)
    intermediate_group.columns = ['_'.join(col).strip() for col in intermediate_group.columns.values]
    intermediate_group.reset_index(inplace=True)
    
    #secondary group by card_id
    final_group = intermediate_group.groupby('card_id').agg(['mean', 'std'])
    final_group.columns = ['_'.join(col).strip() for col in final_group.columns.values]
    final_group.reset_index(inplace=True)
    
    return final_group
#___________________________________________________________
final_group =  aggregate_per_month(historical_transactions) 
final_group[:10]
gc.collect()

CPU times: user 3.61 s, sys: 4 ms, total: 3.62 s
Wall time: 3.61 s


In [31]:
%%time
#merge all the dataframe together
train = pd.merge(train, history, on='card_id', how='left')
test = pd.merge(test, history, on='card_id', how='left')

train = pd.merge(train, authorized, on='card_id', how='left')
test = pd.merge(test, authorized, on='card_id', how='left')

train = pd.merge(train, new, on='card_id', how='left')
test = pd.merge(test, new, on='card_id', how='left')

train = pd.merge(train, final_group, on='card_id', how='left')
test = pd.merge(test, final_group, on='card_id', how='left')

train = pd.merge(train, auth_mean, on='card_id', how='left')
test = pd.merge(test, auth_mean, on='card_id', how='left')

print("Train Shape:", train.shape)
print("Test Shape:", test.shape)
gc.collect()

Train Shape: (201917, 208)
Test Shape: (123623, 208)
CPU times: user 4.38 s, sys: 0 ns, total: 4.38 s
Wall time: 4.4 s


In [32]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time,hist_transactions_count,hist_category_1_sum,hist_category_1_mean,hist_category_2_1.0_mean,hist_category_2_2.0_mean,hist_category_2_3.0_mean,hist_category_2_4.0_mean,hist_category_2_5.0_mean,hist_category_3_A_mean,hist_category_3_B_mean,hist_category_3_C_mean,hist_merchant_id_nunique,hist_2w_purchase_sum,hist_2w_purchase_mean,hist_2w_purchase_max,hist_2w_purchase_min,hist_2w_purchase_std,hist_1m_purchase_sum,hist_1m_purchase_mean,hist_1m_purchase_max,hist_1m_purchase_min,hist_1m_purchase_std,hist_3m_purchase_sum,hist_3m_purchase_mean,hist_3m_purchase_max,hist_3m_purchase_min,hist_3m_purchase_std,hist_6m_purchase_sum,hist_6m_purchase_mean,hist_6m_purchase_max,hist_6m_purchase_min,hist_6m_purchase_std,hist_1y_purchase_sum,hist_1y_purchase_mean,...,new_installments_std,new_purchase_month_mean,new_purchase_month_max,new_purchase_month_min,new_purchase_month_std,new_purchase_date_ptp,new_purchase_date_min,new_purchase_date_max,new_month_lag_min,new_month_lag_max,new_month_lag_std,new_month_lag_sum,month_lag_mean,month_lag_std,purchase_amount_count_mean,purchase_amount_count_std,purchase_amount_sum_mean,purchase_amount_sum_std,purchase_amount_mean_mean,purchase_amount_mean_std,purchase_amount_min_mean,purchase_amount_min_std,purchase_amount_max_mean,purchase_amount_max_std,purchase_amount_std_mean,purchase_amount_std_std,installments_count_mean,installments_count_std,installments_sum_mean,installments_sum_std,installments_mean_mean,installments_mean_std,installments_min_mean,installments_min_std,installments_max_mean,installments_max_std,installments_std_mean,installments_std_std,authorized_flag_sum,authorized_flag_mean
0,2017-06-01,C_ID_92a2005557,5,2,1,245,13.0,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0,0.692308,0.307692,0.000000,12.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-15.189795,-1.168446,0.000000,-13.233655,3.636439,-169.028809,-14.085734,0.000000,-127.007721,36.106964,-169.461578,-14.121799,...,0.000000,3.478261,4.0,3.0,0.510754,4742309.0,1.520259e+09,1.525001e+09,1.0,2.0,0.510754,34.0,-4.500000,1.870829,2.000000,1.095445,-28.243597,59.131435,-8.070309,14.441092,-24.128838,50.635784,-0.918133,0.555219,17.440550,29.070127,2.166667,1.169045,0.666667,0.816497,0.319444,0.423007,0.166667,0.408248,0.5,0.547723,0.269338,0.312603,252.0,0.950943
1,2017-01-01,C_ID_3d0044924f,4,1,0,396,11.0,2.0,0.181818,0.818182,0.0,0.0,0.0,0.0,0.000000,0.363636,0.636364,9.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-60.949657,-5.540878,0.000000,-30.474829,12.327675,-84.938377,-7.721671,4.333439,-30.474829,13.947577,-99.979622,-9.089057,...,0.000000,2.500000,3.0,2.0,0.547723,4887632.0,1.517505e+09,1.522393e+09,1.0,2.0,0.547723,9.0,-5.333333,3.326660,1.833333,1.602082,-16.663271,23.854971,-8.462399,12.403427,-12.166410,14.232433,-6.959726,13.454541,6.293945,8.900982,1.833333,1.602082,7.000000,7.641989,4.033333,3.522310,3.500000,3.728270,5.0,4.289522,1.781853,2.519921,354.0,0.969863
2,2016-08-01,C_ID_d639edf6cd,2,2,0,549,2.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0,1.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.540335,-0.770168,...,NaN,4.000000,4.0,4.0,NaN,0.0,1.524937e+09,1.524937e+09,2.0,2.0,NaN,2.0,-11.000000,1.414214,1.000000,0.000000,-0.770168,0.052010,-0.770168,0.052010,-0.770168,0.052010,-0.770168,0.052010,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,NaN,NaN,42.0,0.954545
3,2017-09-01,C_ID_186d6a6901,4,3,0,153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.755929,3.714286,4.0,3.0,0.487950,3625505.0,1.520424e+09,1.524049e+09,1.0,2.0,0.487950,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [33]:
# train.dropna(inplace = True)
# test.dropna(inplace = True)

In [34]:
#exclude the card_id and first_active month in the features 
features = [c for c in train.columns if c not in ['card_id', 'first_active_month']]
#create a list of categorical features 
categorical_feats = [c for c in features if 'feature_' in c]

In [37]:
train.fillna(train.mean(), inplace=True)
test.fillna(test.mean(), inplace=True)

/opt/conda/lib/python3.6/site-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
/opt/conda/lib/python3.6/site-packages/numpy/core/_methods.py:36: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial)


In [39]:
train.replace([np.inf,-np.inf], 0, inplace=True)
test.replace([np.inf,-np.inf], 0, inplace=True)

In [36]:
train.head()

,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time,hist_transactions_count,hist_category_1_sum,hist_category_1_mean,hist_category_2_1.0_mean,hist_category_2_2.0_mean,hist_category_2_3.0_mean,hist_category_2_4.0_mean,hist_category_2_5.0_mean,hist_category_3_A_mean,hist_category_3_B_mean,hist_category_3_C_mean,hist_merchant_id_nunique,hist_2w_purchase_sum,hist_2w_purchase_mean,hist_2w_purchase_max,hist_2w_purchase_min,hist_2w_purchase_std,hist_1m_purchase_sum,hist_1m_purchase_mean,hist_1m_purchase_max,hist_1m_purchase_min,hist_1m_purchase_std,hist_3m_purchase_sum,hist_3m_purchase_mean,hist_3m_purchase_max,hist_3m_purchase_min,hist_3m_purchase_std,hist_6m_purchase_sum,hist_6m_purchase_mean,hist_6m_purchase_max,hist_6m_purchase_min,hist_6m_purchase_std,hist_1y_purchase_sum,hist_1y_purchase_mean,...,new_installments_std,new_purchase_month_mean,new_purchase_month_max,new_purchase_month_min,new_purchase_month_std,new_purchase_date_ptp,new_purchase_date_min,new_purchase_date_max,new_month_lag_min,new_month_lag_max,new_month_lag_std,new_month_lag_sum,month_lag_mean,month_lag_std,purchase_amount_count_mean,purchase_amount_count_std,purchase_amount_sum_mean,purchase_amount_sum_std,purchase_amount_mean_mean,purchase_amount_mean_std,purchase_amount_min_mean,purchase_amount_min_std,purchase_amount_max_mean,purchase_amount_max_std,purchase_amount_std_mean,purchase_amount_std_std,installments_count_mean,installments_count_std,installments_sum_mean,installments_sum_std,installments_mean_mean,installments_mean_std,installments_min_mean,installments_min_std,installments_max_mean,installments_max_std,installments_std_mean,installments_std_std,authorized_flag_sum,authorized_flag_mean
0,2017-06-01,C_ID_92a2005557,5,2,1,245,13.0,0.0,0.000000,1.000000,0.0,0.0,0.0,0.0,0.692308,0.307692,0.000000,12.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-15.189795,-1.168446,0.000000,-13.233655,3.636439,-169.028809,-14.085734,0.000000,-127.007721,36.106964,-169.461578,-14.121799,...,0.000000,3.478261,4.0,3.0,0.510754,4742309.0,1.520259e+09,1.525001e+09,1.0,2.0,0.510754,34.0,-4.500000,1.870829,2.000000,1.095445,-28.243597,59.131435,-8.070309,14.441092,-24.128838,50.635784,-0.918133,0.555219,17.440550,29.070127,2.166667,1.169045,0.666667,0.816497,0.319444,0.423007,0.166667,0.408248,0.5,0.547723,0.269338,0.312603,252.0,0.950943
1,2017-01-01,C_ID_3d0044924f,4,1,0,396,11.0,2.0,0.181818,0.818182,0.0,0.0,0.0,0.0,0.000000,0.363636,0.636364,9.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-60.949657,-5.540878,0.000000,-30.474829,12.327675,-84.938377,-7.721671,4.333439,-30.474829,13.947577,-99.979622,-9.089057,...,0.000000,2.500000,3.0,2.0,0.547723,4887632.0,1.517505e+09,1.522393e+09,1.0,2.0,0.547723,9.0,-5.333333,3.326660,1.833333,1.602082,-16.663271,23.854971,-8.462399,12.403427,-12.166410,14.232433,-6.959726,13.454541,6.293945,8.900982,1.833333,1.602082,7.000000,7.641989,4.033333,3.522310,3.500000,3.728270,5.0,4.289522,1.781853,2.519921,354.0,0.969863
2,2016-08-01,C_ID_d639edf6cd,2,2,0,549,2.0,0.0,0.000000,0.000000,0.0,0.0,0.0,1.0,1.000000,0.000000,0.000000,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.540335,-0.770168,...,NaN,4.000000,4.0,4.0,NaN,0.0,1.524937e+09,1.524937e+09,2.0,2.0,NaN,2.0,-11.000000,1.414214,1.000000,0.000000,-0.770168,0.052010,-0.770168,0.052010,-0.770168,0.052010,-0.770168,0.052010,NaN,NaN,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,NaN,NaN,42.0,0.954545
3,2017-09-01,C_ID_186d6a6901,4,3,0,153,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.755929,3.714286,4.0,3.0,0.487950,3625505.0,1.520424e+09,1.524049e+09,1.0,2.0,0.487950,12.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [40]:
#perform standardscaler for the train and test dataset 

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
train_scaled = sc.fit_transform(train[features])
test_scaled = sc.fit_transform(test[features])
train_df = pd.DataFrame(train_scaled,columns = train[features].columns)
test_df = pd.DataFrame(test_scaled,columns = test[features].columns)

/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype float32, int64, float16, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype float32, int64, float16, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
/opt/conda/lib/python3.6/site-packages/sklearn/preprocessing/data.py:625: DataConversionWarning: Data with input dtype float32, int64, float16, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
/opt/conda/lib/python3.6/site-packages/sklearn/base.py:462: DataConversionWarning: Data with input dtype float32, int64, float16, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)


In [41]:
train_df['card_id'] = train['card_id']
test_df['card_id'] = test['card_id']
train_df['first_active_month'] = train['first_active_month']
test_df['first_active_month'] = test['first_active_month']

train = train_df
test = test_df

In [42]:
train.shape

(201917, 208)

## LightGBM

In [43]:
#lightgbm configuration
param = {'num_leaves': 50,
         'min_data_in_leaf': 32, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.005,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "nthread": 4,
         "verbosity": -1}

In [ ]:
folds = KFold(n_splits=10, shuffle=True, random_state=15)
oof = np.zeros(len(train))
predictions = np.zeros(len(test))
start = time.time()
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=target.iloc[trn_idx], categorical_feature=categorical_feats)
    val_data = lgb.Dataset(train.iloc[val_idx][features], label=target.iloc[val_idx], categorical_feature=categorical_feats)

    num_round = 100000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 300)
    oof[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions += clf.predict(test[features], num_iteration=clf.best_iteration) / folds.n_splits

print("CV score: {:<8.5f}".format(mean_squared_error(oof, target)**0.5))

In [ ]:
cols = (feature_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')

## LightGBM-1 with Repeated kfold approach

#### RepeatedKFold repeats K-Fold n times. It can be used when one requires to run KFold n times, producing different splits in each repetition.

In [ ]:
lgbparam = {'num_leaves': 31,
#             'boosting_type': 'rf',
             'min_data_in_leaf': 30, 
             'objective':'regression',
             'max_depth': 12,
             'learning_rate': 0.01,
             "min_child_samples": 20,
             "boosting": "gbdt",
             "feature_fraction": 0.9,
             "bagging_freq": 1,
             "bagging_fraction": 0.9 ,
             "bagging_seed": 11,
             "metric": 'rmse',
             "lambda_l1": 0.1,
             "verbosity": -1,
             "nthread": 4,
             "random_state": 4590}

In [ ]:
from sklearn.model_selection import RepeatedKFold
folds = RepeatedKFold(n_splits=10, n_repeats=2, random_state=4520)

oof_lgb = np.zeros(len(train))
predictions_lgb = np.zeros(len(test))
start = time.time()
feature_importance_df = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train.values, target.values)):
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][features], label=target.iloc[trn_idx], categorical_feature=categorical_feats)
    val_data = lgb.Dataset(train.iloc[val_idx][features], label=target.iloc[val_idx], categorical_feature=categorical_feats)

    num_round = 100000
    clf = lgb.train(lgbparam, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=3000, early_stopping_rounds = 300)
    oof_lgb[val_idx] = clf.predict(train.iloc[val_idx][features], num_iteration=clf.best_iteration)
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = features
    fold_importance_df["importance"] = clf.feature_importance()
    fold_importance_df["fold"] = fold_ + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    
    predictions_lgb += clf.predict(test[features], num_iteration=clf.best_iteration) / (5 * 2)

print("CV score: {:<8.5f}".format(mean_squared_error(oof_lgb, target)**0.5))

In [ ]:
cols = (feature_importance_df[["feature", "importance"]]
        .groupby("feature")
        .mean()
        .sort_values(by="importance", ascending=False)[:1000].index)

best_features = feature_importance_df.loc[feature_importance_df.feature.isin(cols)]

plt.figure(figsize=(14,25))
sns.barplot(x="importance",
            y="feature",
            data=best_features.sort_values(by="importance",
                                           ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.savefig('lgbm_importances.png')

In [ ]:
sub_df = pd.DataFrame({"card_id":test["card_id"].values})
sub_df["target"] = predictions
sub_df.to_csv("submit_lgb.csv", index=False)

sub_df1 = pd.DataFrame({"card_id":test["card_id"].values})
sub_df1["target"] = predictions_lgb
sub_df1.to_csv("submit_lgb1.csv", index=False)

## Stacking

In [ ]:
train_stack = np.vstack([oof,oof_lgb]).transpose()
test_stack = np.vstack([predictions,predictions_lgb]).transpose()

folds = RepeatedKFold(n_splits=5,n_repeats=1,random_state=4520)
oof_stack = np.zeros(train_stack.shape[0])
predictions_stack = np.zeros(test_stack.shape[0])

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train_stack, target)):
    print("fold n°{}".format(fold_))
    trn_data, trn_y = train_stack[trn_idx], target.iloc[trn_idx].values
    val_data, val_y = train_stack[val_idx], target.iloc[val_idx].values

    print("-" * 10 + "Stacking " + str(fold_) + "-" * 10)
#     cb_model = CatBoostRegressor(iterations=3000, learning_rate=0.1, depth=8, l2_leaf_reg=20, bootstrap_type='Bernoulli',  eval_metric='RMSE', metric_period=50, od_type='Iter', od_wait=45, random_seed=17, allow_writing_files=False)
#     cb_model.fit(trn_data, trn_y, eval_set=(val_data, val_y), cat_features=[], use_best_model=True, verbose=True)
    clf = BayesianRidge()
    clf.fit(trn_data, trn_y)
    
    oof_stack[val_idx] = clf.predict(val_data)
    predictions_stack += clf.predict(test_stack) / 5


np.sqrt(mean_squared_error(target.values, oof_stack))

In [ ]:
sample_submission = pd.read_csv('../input/elo-merchant-category-recommendation/sample_submission.csv')
sample_submission['target'] = predictions_stack
sample_submission.to_csv('Bayesian_Ridge_Stacking.csv', index=False)

In [ ]:
sample_submission = pd.read_csv('../input/elo-merchant-category-recommendation/sample_submission.csv')
sample1 = pd.read_csv("../input/elo-blending/3.695.csv")
sample2 = pd.read_csv("../input/elo-blending/combining_submission (1).csv")
sample_submission['target'] = predictions * 0.5 + predictions_lgb * 0.5
sample_submission.to_csv("Blend1.csv", index = False)
sample_submission['target'] = sample_submission['target'] * 0.2 + sample1['target'] * 0.2 + sample2['target'] * 0.6
sample_submission.to_csv('Blend2.csv', index=False)